In [1]:
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
import holidays

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression

from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# Step 1: Data Processing

In [3]:
# Load data
data = pd.read_excel('pollstarKPOP_youtube_spotify.xlsx')
data.head(3)

,Unnamed: 0,Event Date,# Shows,Headliner,Support,Venue,City,State,Country,Market,...,yt Title,yt Description,yt Published At,yt View Count,yt Subscriber Count,yt Video Count,sp artist_name,sp artist_genre,sp followers,sp popularity
0,1,2023-11-04,1,xikers,NaN,Warfield Theatre,San Francisco,California,United States,San Francisco-Oakland-San Jose,...,xikers,xikers(싸이커스) Official YouTube Channel\n,2022-11-15T14:21:56.347452Z,37578254.0,375000.0,846.0,xikers,['k-pop boy group'],332220,50
1,2,2023-11-03,1,EVERGLOW,NaN,Kings Theatre,Brooklyn,New York,United States,New York,...,Everglow,"Welcome to my channel! \nMy name is Marco, I'm...",2012-07-18T21:07:58Z,28337056.0,110000.0,554.0,EVERGLOW,"['k-pop', 'k-pop girl group']",2098681,59
2,3,2023-11-01,1,Xikers,NaN,Orpheum Theatre,Los Angeles,California,United States,Los Angeles,...,xikers,xikers(싸이커스) Official YouTube Channel\n,2022-11-15T14:21:56.347452Z,37578254.0,375000.0,846.0,Xikers,['k-pop boy group'],332220,50


In [4]:
race_data=pd.read_csv('Race_USCensusBureau.csv')
race_data.head(3)

,State,Total:,Population of one race:,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Population of two or more races:,...,American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Population of five races:,White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race,White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race,White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Population of six races:,White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race
0,Alabama,5024279.0,4767326.0,3220452.0,1296162.0,33625.0,76660.0,2984.0,137443.0,256953.0,...,4.0,187.0,89.0,78.0,13.0,0.0,5.0,2.0,9.0,9.0
1,Alaska,733391.0,643867.0,435392.0,21898.0,111575.0,44032.0,12698.0,18272.0,89524.0,...,11.0,97.0,44.0,29.0,4.0,4.0,14.0,2.0,13.0,13.0
2,Arizona,7151502.0,6154696.0,4322337.0,339150.0,319512.0,257430.0,16397.0,899870.0,996806.0,...,18.0,440.0,174.0,215.0,12.0,7.0,28.0,4.0,70.0,70.0


In [5]:
# Extra Features 1: day of the week

# Convert the date column to datetime format if it's not already in datetime format
data['Event Date'] = pd.to_datetime(data['Event Date'])
# Extract the day of the week (0 = Monday, 1 = Tuesday, ..., 6 = Sunday)
data['day_of_week'] = data['Event Date'].dt.dayofweek

In [6]:
# Extra feature 2: Holiday or Not
def is_holiday(date):
    us_holidays = holidays.US()
    return date in us_holidays
data['is_holiday'] = data['Event Date'].apply(is_holiday)

In [7]:
# Extra feature 3: Asian Population in the state
# Merge dataframes based on the 'State' column
data = pd.merge(data, race_data.iloc[:, [0, 1, 6]], on='State')
last_column_name = data.columns[-1]
data = data.rename(columns={last_column_name: 'Asian alone population'})
last2_column_name = data.columns[-2]
data = data.rename(columns={last2_column_name: 'Total population'})
data['Asian percentage'] = (data['Asian alone population'] / data['Total population']) * 100


In [8]:
# Extra fearure 4: GDP data

In [9]:
# check data
data.head(3)

,Unnamed: 0,Event Date,# Shows,Headliner,Support,Venue,City,State,Country,Market,...,yt Video Count,sp artist_name,sp artist_genre,sp followers,sp popularity,day_of_week,is_holiday,Total population,Asian alone population,Asian percentage
0,1,2023-11-04,1,xikers,NaN,Warfield Theatre,San Francisco,California,United States,San Francisco-Oakland-San Jose,...,846.0,xikers,['k-pop boy group'],332220,50,5,False,39538223.0,6085947.0,15.392566
1,3,2023-11-01,1,Xikers,NaN,Orpheum Theatre,Los Angeles,California,United States,Los Angeles,...,846.0,Xikers,['k-pop boy group'],332220,50,2,False,39538223.0,6085947.0,15.392566
2,13,2023-10-14,1,tripleS,NaN,Pasadena Civic Auditorium,Pasadena,California,United States,Los Angeles,...,1404.0,tripleS,['k-pop girl group'],334787,56,5,False,39538223.0,6085947.0,15.392566


In [10]:
# Define feature combinations to test
feature_combinations = [
    ['Location','sp followers', 'yt Subscriber Count'],
    ['Location','sp followers', 'yt Subscriber Count','Month'],
    ['Location','sp followers', 'Market'],
    ['Location','Venue','Avg. Event Capacity','Month'],
    ['Location','sp followers', 'Avg. Event Capacity','Month','Asian alone population','is_holiday'],
    ['Location','sp followers', 'yt Subscriber Count','Month','Market','Venue','Avg. Event Capacity','Asian alone population','Asian percentage','is_holiday','day_of_week']
    # Add more feature combinations as needed
]

In [11]:
# check the avaiablity of each features
# List of columns for which you want to calculate the percentage of non-null values
columns_to_check = ['Location',
                    'sp followers', 
                    'yt Subscriber Count',
                    'Month','Market',
                    'Venue',
                    'Avg. Event Capacity',
                    'Asian alone population',
                    'Asian percentage',
                    'is_holiday',
                    'day_of_week'
                   ]

# Calculate the percentage of non-null values for each column
percentages = {}
for column in columns_to_check:
    percentage = (data[column].count() / len(data)) * 100
    percentages[column] = percentage

# Convert the dictionary to a DataFrame
result_df = pd.DataFrame.from_dict(percentages, orient='index', columns=['Percentage'])
result_df.index.name = 'Column'

print(result_df)

                        Percentage
Column                            
Location                100.000000
sp followers            100.000000
yt Subscriber Count      91.129032
Month                   100.000000
Market                   98.655914
Venue                   100.000000
Avg. Event Capacity     100.000000
Asian alone population  100.000000
Asian percentage        100.000000
is_holiday              100.000000
day_of_week             100.000000


In [12]:
# Drop rows with null values in the selected columns
columns_to_drop=['Location','sp followers', 'yt Subscriber Count','Month','Market','Venue','Avg. Event Capacity']
data.dropna(subset=columns_to_drop, inplace=True)

In [14]:
# export as csv
data.to_csv('PlusSupplementaryFeatures.csv', index=True)